In [1]:
import pandas as pd
import json
from thefuzz import process
from thefuzz import fuzz
from tqdm import tqdm

/home/pinbo/Documents/tri-nit-ml3/.venv/lib64/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
with open('data/city.list.json', 'r') as file:
    data = json.load(file)

In [3]:
# Filter the JSON objects with "country": "IN"
city_list = [d for d in data if d['country'] == 'IN']

# Convert the filtered JSON objects into a DataFrame
city_df = pd.DataFrame(city_list)
city_df

,id,name,state,country,coord
0,1163626.0,Thang,,IN,"{'lon': 76.75, 'lat': 34.916672}"
1,1167718.0,Pūnch,,IN,"{'lon': 74.089737, 'lat': 33.767879}"
2,1252646.0,Keelakarai,,IN,"{'lon': 78.783333, 'lat': 9.23333}"
3,1252653.0,Zunheboto,,IN,"{'lon': 94.51667, 'lat': 25.966669}"
4,1252668.0,Ziro,,IN,"{'lon': 93.833328, 'lat': 27.633329}"
...,...,...,...,...,...
3629,8131500.0,Nagpur Division,,IN,"{'lon': 79.107483, 'lat': 21.149111}"
3630,8131501.0,Nashik Division,,IN,"{'lon': 73.789421, 'lat': 19.98295}"
3631,8131502.0,Satara Division,,IN,"{'lon': 73.859283, 'lat': 18.51787}"
3632,8199371.0,Rambagh,,IN,"{'lon': 75.808319, 'lat': 26.90266}"


In [4]:
rainfall_df = pd.read_csv('./data/district wise rainfall normal.csv')
rainfall_df

,STATE_UT_NAME,DISTRICT,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec
0,ANDAMAN And NICOBAR ISLANDS,NICOBAR,107.3,57.9,65.2,117.0,358.5,295.5,285.0,271.9,354.8,326.0,315.2,250.9,2805.2,165.2,540.7,1207.2,892.1
1,ANDAMAN And NICOBAR ISLANDS,SOUTH ANDAMAN,43.7,26.0,18.6,90.5,374.4,457.2,421.3,423.1,455.6,301.2,275.8,128.3,3015.7,69.7,483.5,1757.2,705.3
2,ANDAMAN And NICOBAR ISLANDS,N & M ANDAMAN,32.7,15.9,8.6,53.4,343.6,503.3,465.4,460.9,454.8,276.1,198.6,100.0,2913.3,48.6,405.6,1884.4,574.7
3,ARUNACHAL PRADESH,LOHIT,42.2,80.8,176.4,358.5,306.4,447.0,660.1,427.8,313.6,167.1,34.1,29.8,3043.8,123.0,841.3,1848.5,231.0
4,ARUNACHAL PRADESH,EAST SIANG,33.3,79.5,105.9,216.5,323.0,738.3,990.9,711.2,568.0,206.9,29.5,31.7,4034.7,112.8,645.4,3008.4,268.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,KERALA,IDUKKI,13.4,22.1,43.6,150.4,232.6,651.6,788.9,527.3,308.4,343.2,172.9,48.1,3302.5,35.5,426.6,2276.2,564.2
637,KERALA,KASARGOD,2.3,1.0,8.4,46.9,217.6,999.6,1108.5,636.3,263.1,234.9,84.6,18.4,3621.6,3.3,272.9,3007.5,337.9
638,KERALA,PATHANAMTHITTA,19.8,45.2,73.9,184.9,294.7,556.9,539.9,352.7,266.2,359.4,213.5,51.3,2958.4,65.0,553.5,1715.7,624.2
639,KERALA,WAYANAD,4.8,8.3,17.5,83.3,174.6,698.1,1110.4,592.9,230.7,213.1,93.6,25.8,3253.1,13.1,275.4,2632.1,332.5


In [5]:
# Create a function to perform the fuzzy match
def fuzzy_match(city, district):
    return fuzz.token_sort_ratio(city, district)

In [ ]:
# Initialize an empty list to store the matched coord values
coords = []

# Loop over the rows of the rainfall_df DataFrame
for index, district_match in tqdm(rainfall_df[:20].iterrows(),total=len(rainfall_df)):
    # Initialize a variable to store the maximum match ratio
    coord = None
    district = district_match['DISTRICT']
    # print(district)

    choices = city_df.name.values
    # print(choices)

    ratios = process.extract(district, choices, limit=1,scorer=fuzz.partial_token_sort_ratio)
    # print(ratios[0][0])

    print(f"Match: {ratios[0][0]} | Actual: {district}")
    
    district_match = city_df[city_df['name'] == ratios[0][0]]


    # Add the matched coord value to the list
    coords.append(district_match.coord.values[0])

# Add the list of coord values to the rainfall_df DataFrame
rainfall_df['coord'] = coords

# Print the updated rainfall_df DataFrame
print(rainfall_df)


In [7]:
rainfall_df

,STATE_UT_NAME,DISTRICT,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec,coord
0,ANDAMAN And NICOBAR ISLANDS,NICOBAR,107.3,57.9,65.2,117.0,358.5,295.5,285.0,271.9,354.8,326.0,315.2,250.9,2805.2,165.2,540.7,1207.2,892.1,"{'lon': 92.675171, 'lat': 11.70065}"
1,ANDAMAN And NICOBAR ISLANDS,SOUTH ANDAMAN,43.7,26.0,18.6,90.5,374.4,457.2,421.3,423.1,455.6,301.2,275.8,128.3,3015.7,69.7,483.5,1757.2,705.3,"{'lon': 72.849998, 'lat': 20.41667}"
2,ANDAMAN And NICOBAR ISLANDS,N & M ANDAMAN,32.7,15.9,8.6,53.4,343.6,503.3,465.4,460.9,454.8,276.1,198.6,100.0,2913.3,48.6,405.6,1884.4,574.7,"{'lon': 72.849998, 'lat': 20.41667}"
3,ARUNACHAL PRADESH,LOHIT,42.2,80.8,176.4,358.5,306.4,447.0,660.1,427.8,313.6,167.1,34.1,29.8,3043.8,123.0,841.3,1848.5,231.0,"{'lon': 77.066673, 'lat': 28.683331}"
4,ARUNACHAL PRADESH,EAST SIANG,33.3,79.5,105.9,216.5,323.0,738.3,990.9,711.2,568.0,206.9,29.5,31.7,4034.7,112.8,645.4,3008.4,268.1,"{'lon': 92.666672, 'lat': 23.799999}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,KERALA,IDUKKI,13.4,22.1,43.6,150.4,232.6,651.6,788.9,527.3,308.4,343.2,172.9,48.1,3302.5,35.5,426.6,2276.2,564.2,"{'lon': 76.966667, 'lat': 9.85}"
637,KERALA,KASARGOD,2.3,1.0,8.4,46.9,217.6,999.6,1108.5,636.3,263.1,234.9,84.6,18.4,3621.6,3.3,272.9,3007.5,337.9,"{'lon': 76.283333, 'lat': 24.91667}"
638,KERALA,PATHANAMTHITTA,19.8,45.2,73.9,184.9,294.7,556.9,539.9,352.7,266.2,359.4,213.5,51.3,2958.4,65.0,553.5,1715.7,624.2,"{'lon': 76.783333, 'lat': 9.26667}"
639,KERALA,WAYANAD,4.8,8.3,17.5,83.3,174.6,698.1,1110.4,592.9,230.7,213.1,93.6,25.8,3253.1,13.1,275.4,2632.1,332.5,"{'lon': 75.050003, 'lat': 22.033331}"


In [22]:
rainfall_df.to_csv('./data/rainfall_lat_long_fuzzy.csv')